In [186]:
#pip install progressbar
#pip install XXXXXXXXXXX [other packages needed]

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import sys
import ast
import string
import missingno as msno 
import seaborn as sn
import requests
import datetime
import progressbar
import time
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from pandas_profiling import ProfileReport
from pandas.plotting import scatter_matrix
from matplotlib import pyplot

#Maybe not used! (REVIEW)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn import cluster

from sklearn.neighbors import kneighbors_graph

import warnings
warnings.filterwarnings("ignore")

from sklearn.impute import KNNImputer


# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', size=12) 
plt.rc('figure', figsize = (12, 5))

# Settings for the visualizations
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

## 5.1 Import Data

In [4]:
df_golden_record_raw = pd.read_excel('Clean_data/Golden_Record.xlsx',index_col=0)

In [5]:
df_golden_record = df_golden_record_raw.copy()
df_golden_record

,games_started_RegS,field_goals_attempts_pg_RegS,field_goals_percent_pg_RegS,_3pts_goals_attempts_pg_RegS,FT_goals_attempts_pg_RegS,FT_goals_percent_pg_RegS,total_rebounds_pg_RegS,assists_pg_RegS,personal_foults_pg_RegS,games_started_Playoff,...,draft_year,draft_round,draft_pick_number,n_playoff,n_total_wins,w_l_pct,most_popular_team_played,triple_double_metric_RegS,triple_double_metric_Playoff,championship_pct
0,3,3.0,0.434,0.1,0.6,0.672,2.4,1.3,1.7,0.0,...,1978,5,104,6,744,0.504,Washington Wizards,0.223333,0.010000,0.0
1,0,3.5,0.456,-1.0,1.2,0.774,0.6,1.4,1.1,0.0,...,1968,1,101,4,192,0.585,Atlanta Hawks,0.203333,0.000000,0.0
2,11,2.4,0.444,0.1,0.7,0.743,2.0,0.4,2.3,0.0,...,1994,1,22,0,51,0.207,Golden State Warriors,0.170000,0.000000,0.0
3,2,2.8,0.433,0.0,1.0,0.613,2.4,0.3,1.2,0.0,...,1991,1,22,1,42,0.512,Miami Heat,0.190000,0.000000,0.0
4,11,5.0,0.507,0.0,1.1,0.720,3.2,0.6,1.4,0.0,...,2008,1,29,2,86,0.524,Charlotte Hornets,0.323333,0.066667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3437,0,2.6,0.286,0.0,0.3,1.000,1.8,0.1,1.6,NaN,...,1979,2,32,0,37,0.451,Indiana Pacers,0.123333,NaN,0.0
3438,23,2.9,0.408,0.0,1.2,0.783,2.1,0.2,1.8,NaN,...,1995,1,22,2,122,0.744,Oklahoma City Thunder,0.190000,NaN,0.0
3439,0,1.5,0.667,0.0,0.0,0.000,2.0,3.5,2.0,NaN,...,2003,2,40,4,196,0.598,Brooklyn Nets,0.250000,NaN,0.0
3440,0,1.6,0.323,0.0,0.3,0.600,1.8,0.2,0.9,NaN,...,2016,2,41,0,29,0.354,Orlando Magic,0.106667,NaN,0.0


## 5. 2 xxxxx

### Standarize all the transformations with pipelines, in order to get a productivized model (hacer en notebook a parte, guardate unos pocos -> 10 jugadores)

In [64]:
#log + StandScales necessary!!!

In [65]:
num_pipeline_playoff = Pipeline([
        ('KNNimputer', KNNImputer(n_neighbors=30,weights='distance')), #if there is any NaN
        #('log',FunctionTransformer(np.log1p, validate=True)), #to smooth the outliers
        ('std_scaler', StandardScaler())
    ])
num_pipeline_no_playoff = Pipeline([
        ('KNNimputer', KNNImputer(n_neighbors=30,weights='distance')), #if there is any NaN
        #('log',FunctionTransformer(np.log1p, validate=True)), #to smooth the outliers
        ('std_scaler', StandardScaler())
    ])

In [66]:
def BinaryToNumber(X):
    result = X.copy()
    
    result[result=='Yes'] = 1
    result[result=='No'] = 0
    
    return result

In [67]:
binary_pipeline = Pipeline([
        ('SimpleImputer',SimpleImputer(strategy="most_frequent")),
        ('BinaryToNumber',FunctionTransformer(func = BinaryToNumber,validate=False))
    ])

In [68]:
def Standarize_Positions(X):
    result = X.copy()
    
    result[result=='F-G'] = 'G-F'
    result[result=='C-F'] = 'F-C'
    
    #result = result.apply(lambda x: 'G-F' if x=='F-G' else 'F-C' if x=='C-F' else x)
    
    return result

In [69]:
def Standarize_Teams(X):
    result = X.copy()
    
    top_10_list = ['Philadelphia 76ers','Golden State Warriors','Brooklyn Nets','Atlanta Hawks','Detroit Pistons',
                   'Sacramento Kings','New York Knicks','Washington Wizards','Los Angeles Clippers','Los Angeles Lakers']
    result[np.isin(result,top_10_list)==False] = 'Other'
    
    return result

In [70]:
#OHE_pipeline = ColumnTransformer(transformers=[('imputer', SimpleImputer(strategy="constant",fill_value='Unknown'),[0,1]),
#                                               ('Standarize_Positions',FunctionTransformer(func = Standarize_Positions,validate=False),[0]),
#                                               ('Standarize_Teams',FunctionTransformer(func = Standarize_Teams,validate=False),[1]),
#                                               ('cat', OneHotEncoder(handle_unknown='ignore'), [0, 1])])
#NOT WORKING!?

In [71]:
OHE_pipeline_position = Pipeline([('imputer', SimpleImputer(strategy="constant",fill_value='Unknown')),
                                  ('Standarize_Positions',FunctionTransformer(func = Standarize_Positions,validate=False)),
                                  ('cat', OneHotEncoder())])

In [72]:
OHE_pipeline_teams = Pipeline([('imputer', SimpleImputer(strategy="constant",fill_value='Unknown')),
                                  ('Standarize_Teams',FunctionTransformer(func = Standarize_Teams,validate=False)),
                                  ('cat', OneHotEncoder())])

In [73]:
#per separat funciona!!!!

We firstly clear (of columns we won't use in the model) and separate the Dataframes

In [85]:
cols_used = ['games_started_RegS', 'field_goals_attempts_pg_RegS',
       'field_goals_percent_pg_RegS', '_3pts_goals_attempts_pg_RegS',
       'FT_goals_attempts_pg_RegS', 'FT_goals_percent_pg_RegS',
       'total_rebounds_pg_RegS', 'assists_pg_RegS', 'personal_foults_pg_RegS',
       'games_started_Playoff', 'field_goals_attempts_pg_Playoff',
       'field_goals_percent_pg_Playoff', '_3pts_goals_attempts_pg_Playoff',
       'FT_goals_attempts_pg_Playoff', 'FT_goals_percent_pg_Playoff',
       'total_rebounds_pg_Playoff', 'assists_pg_Playoff',
       'personal_foults_pg_Playoff', 'FT_goals_attempts_total_RegS',
       'assists_total_RegS', 'FT_goals_attempts_total_Playoff',
       'assists_total_Playoff', 'player_eff_rating_RegS',
       'true_shooting_eff_RegS', 'usage_percent_RegS', 'win_share_RegS',
       'player_eff_rating_Playoff', 'true_shooting_eff_Playoff',
       'usage_percent_Playoff', 'win_share_Playoff', 'Hall_Of_Fame',
       'n_NBA_Champ', 'n_All_Star', 'gone_to_college', 'n_teams', 'active_from', 'active_to', 'weight',
       'birth_date(year)', 'height(cm)', 'age_entrance', 'age_retirement',
       'years_active', 'draft_year', 'draft_round', 'draft_pick_number',
       'n_playoff', 'n_total_wins', 'w_l_pct', 'triple_double_metric_RegS',
       'triple_double_metric_Playoff', 'championship_pct','Hall_Of_Fame','gone_to_college','position',
       'most_popular_team_played','Playoff_flag']

df_golden_record = df_golden_record_raw[[c for c in df_golden_record_raw.columns if c in cols_used]]

df_golden_record_playoff = df_golden_record[df_golden_record['Playoff_flag']=='Yes'].drop(columns=['Playoff_flag'])
df_golden_record_no_playoff = df_golden_record[df_golden_record['Playoff_flag']=='No'].drop(columns=['Playoff_flag'])
df_golden_record_no_playoff = df_golden_record_no_playoff.dropna(axis=1)

In [75]:
df_golden_record_no_playoff.head()

,games_started_RegS,field_goals_attempts_pg_RegS,field_goals_percent_pg_RegS,_3pts_goals_attempts_pg_RegS,FT_goals_attempts_pg_RegS,FT_goals_percent_pg_RegS,total_rebounds_pg_RegS,assists_pg_RegS,personal_foults_pg_RegS,FT_goals_attempts_total_RegS,...,years_active,draft_year,draft_round,draft_pick_number,n_playoff,n_total_wins,w_l_pct,most_popular_team_played,triple_double_metric_RegS,championship_pct
2316,0,2.0,0.000,-1.0,0.0,0.000,1.0,1.0,1.0,0,...,0,1956,3,21,2,76,0.528,Philadelphia 76ers,0.066667,0.0
2317,0,3.1,0.370,0.8,0.3,0.500,1.0,0.5,0.4,10,...,3,2005,2,60,0,57,0.232,Los Angeles Clippers,0.140000,0.0
2318,60,3.9,0.444,1.6,1.1,0.759,3.5,0.6,2.0,386,...,6,2012,2,37,0,19,0.232,Phoenix Suns,0.300000,0.0
2319,0,2.0,0.314,0.0,0.4,0.700,2.3,0.2,0.6,10,...,1,2010,2,50,0,45,0.308,Toronto Raptors,0.133333,0.0
2320,0,4.5,0.293,-1.0,1.2,0.598,4.2,0.3,2.2,92,...,1,1959,3,106,4,144,0.456,Los Angeles Lakers,0.260000,0.0


In [76]:
df_golden_record_playoff.head()

,games_started_RegS,field_goals_attempts_pg_RegS,field_goals_percent_pg_RegS,_3pts_goals_attempts_pg_RegS,FT_goals_attempts_pg_RegS,FT_goals_percent_pg_RegS,total_rebounds_pg_RegS,assists_pg_RegS,personal_foults_pg_RegS,games_started_Playoff,...,draft_year,draft_round,draft_pick_number,n_playoff,n_total_wins,w_l_pct,most_popular_team_played,triple_double_metric_RegS,triple_double_metric_Playoff,championship_pct
0,3,3.0,0.434,0.1,0.6,0.672,2.4,1.3,1.7,0.0,...,1978,5,104,6,744,0.504,Washington Wizards,0.223333,0.010000,0.0
1,0,3.5,0.456,-1.0,1.2,0.774,0.6,1.4,1.1,0.0,...,1968,1,101,4,192,0.585,Atlanta Hawks,0.203333,0.000000,0.0
2,11,2.4,0.444,0.1,0.7,0.743,2.0,0.4,2.3,0.0,...,1994,1,22,0,51,0.207,Golden State Warriors,0.170000,0.000000,0.0
3,2,2.8,0.433,0.0,1.0,0.613,2.4,0.3,1.2,0.0,...,1991,1,22,1,42,0.512,Miami Heat,0.190000,0.000000,0.0
4,11,5.0,0.507,0.0,1.1,0.720,3.2,0.6,1.4,0.0,...,2008,1,29,2,86,0.524,Charlotte Hornets,0.323333,0.066667,0.0


In [99]:
binary_attribs = ['Hall_Of_Fame','gone_to_college']

num_attribs_playoff = ['games_started_RegS', 'field_goals_attempts_pg_RegS',
       'field_goals_percent_pg_RegS', '_3pts_goals_attempts_pg_RegS',
       'FT_goals_attempts_pg_RegS', 'FT_goals_percent_pg_RegS',
       'total_rebounds_pg_RegS', 'assists_pg_RegS', 'personal_foults_pg_RegS',
       'games_started_Playoff', 'field_goals_attempts_pg_Playoff',
       'field_goals_percent_pg_Playoff', '_3pts_goals_attempts_pg_Playoff',
       'FT_goals_attempts_pg_Playoff', 'FT_goals_percent_pg_Playoff',
       'total_rebounds_pg_Playoff', 'assists_pg_Playoff',
       'personal_foults_pg_Playoff', 'FT_goals_attempts_total_RegS',
       'assists_total_RegS', 'FT_goals_attempts_total_Playoff',
       'assists_total_Playoff', 'player_eff_rating_RegS',
       'true_shooting_eff_RegS', 'usage_percent_RegS', 'win_share_RegS',
       'player_eff_rating_Playoff', 'true_shooting_eff_Playoff',
       'usage_percent_Playoff', 'win_share_Playoff',
       'n_NBA_Champ', 'n_All_Star', 'n_teams', 'active_from', 'active_to', 'weight',
       'birth_date(year)', 'height(cm)', 'age_entrance', 'age_retirement',
       'years_active', 'draft_year', 'draft_round', 'draft_pick_number',
       'n_playoff', 'n_total_wins', 'w_l_pct', 'triple_double_metric_RegS',
       'triple_double_metric_Playoff', 'championship_pct']

num_attribs_no_playoff = ['games_started_RegS', 'field_goals_attempts_pg_RegS',
       'field_goals_percent_pg_RegS', '_3pts_goals_attempts_pg_RegS',
       'FT_goals_attempts_pg_RegS', 'FT_goals_percent_pg_RegS',
       'total_rebounds_pg_RegS', 'assists_pg_RegS', 'personal_foults_pg_RegS',
       'FT_goals_attempts_total_RegS', 'assists_total_RegS',
       'player_eff_rating_RegS', 'true_shooting_eff_RegS',
       'usage_percent_RegS', 'win_share_RegS', 'n_NBA_Champ',
       'n_All_Star', 'n_teams',
       'active_from', 'active_to', 'weight', 'birth_date(year)', 'height(cm)',
       'age_entrance', 'age_retirement', 'years_active', 'draft_year',
       'draft_round', 'draft_pick_number', 'n_playoff', 'n_total_wins',
       'w_l_pct', 'triple_double_metric_RegS', 'championship_pct']

cat_attribs_postion = ['position']
cat_attribs_teams = ['most_popular_team_played']

full_pipeline_playoff = ColumnTransformer([
        ("num", num_pipeline_playoff, num_attribs_playoff),
        ("binary", binary_pipeline, binary_attribs),
        ("cat_teams", OHE_pipeline_teams, cat_attribs_teams),
        ("cat_positions", OHE_pipeline_position, cat_attribs_postion)
    ])

full_pipeline_no_playoff = ColumnTransformer([
        ("num", num_pipeline_no_playoff, num_attribs_no_playoff),
        ("binary", binary_pipeline, binary_attribs),
        ("cat_teams", OHE_pipeline_teams, cat_attribs_teams),
        ("cat_positions", OHE_pipeline_position, cat_attribs_postion)
    ])

In [100]:
playoff_prepared = full_pipeline_playoff.fit_transform(df_golden_record_playoff)
playoff_prepared

array([[-0.673386491753755, -1.1063264403102047, -0.18552324323493277,
        ..., 0.0, 0.0, 1.0],
       [-0.6848214964456324, -0.9819983845399302, 0.19596028270664634,
        ..., 0.0, 1.0, 0.0],
       [-0.6428931459087487, -1.255520107234534, -0.012121640534214984,
        ..., 0.0, 0.0, 0.0],
       ...,
       [3.2755017951745673, 3.8419301793467215, 0.9069068537795882, ...,
        0.0, 0.0, 1.0],
       [-0.6848214964456324, 2.4743215658737014, -0.11616260215464565,
        ..., 0.0, 1.0, 0.0],
       [2.749491579348208, 3.5684084566521177, -0.34158468566557876, ...,
        0.0, 1.0, 0.0]], dtype=object)

In [103]:
no_playoff_prepared = full_pipeline_no_playoff.fit_transform(df_golden_record_no_playoff)
no_playoff_prepared

array([[-0.3298722617237494, -0.6949675907482202, -2.9692427299149293,
        ..., 0.0, 1.0, 0.0],
       [-0.3298722617237494, -0.2953450940440672, -0.15333971417159323,
        ..., 0.0, 1.0, 0.0],
       [1.1168119318534337, -0.004710550986501419, 0.4098408889770741,
        ..., 1.0, 0.0, 0.0],
       ...,
       [-0.3298722617237494, -0.8766141801591989, 2.1069932471142745,
        ..., 0.0, 1.0, 0.0],
       [-0.3298722617237494, -0.8402848622770032, -0.5110355026579089,
        ..., 0.0, 0.0, 0.0],
       [-0.3298722617237494, -0.5133210013372417, -0.20661355500998071,
        ..., 0.0, 1.0, 0.0]], dtype=object)

In [ ]:
# a = pd.DataFrame(housing_prepared, columns=housing.columns,
#                           index=housing.index)

In [ ]:
#per lligar amb el seguent problema, fer un gràfic dels clusters per any del draft!